In [4]:
source("Code/importa_dados.R", echo = T)


> customers <- read.csv("Dados/olist_customers_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> geolocation <- read.csv("Dados/olist_geolocation_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> items <- read.csv("Dados/olist_order_items_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> payments <- read.csv("Dados/olist_order_payments_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> reviews <- read.csv("Dados/olist_order_reviews_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> orders <- read.csv("Dados/olist_orders_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> products <- read.csv("Dados/olist_products_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> sellers <- read.csv("Dados/olist_sellers_dataset.csv", 
+     encoding = "UTF-8", stringsAsFactors = F)

> product_category <- read.csv("Dados/product_category_name_translation.csv", 
+     encoding = "UTF-8", stringsAsF

## Ativação dos pacotes necessários:

In [7]:
require(tidyverse)
require(data.table)
require(arules)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: arules
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand


Attaching package: ‘arules’

The following object is masked from ‘package:dplyr’:

    recode

The following objects are masked from ‘package:base’:

    abbreviate, write



## Clientes

In [10]:
glimpse(customers)

customers %>% 
  count(customer_id, sort = T) %>% 
  filter(n > 1)

customers %>% 
  count(customer_unique_id, sort = T) %>% 
  filter(n > 1)

customers %>% 
  count(customer_state, sort = T)

Observations: 99,441
Variables: 5
$ customer_id              <chr> "06b8999e2fba1a1fbc88172c00ba8bc7", "18955e8…
$ customer_unique_id       <chr> "861eff4711a542e4b93843c6dd7febb0", "290c77b…
$ customer_zip_code_prefix <int> 14409, 9790, 1151, 8775, 13056, 89254, 4534,…
$ customer_city            <chr> "franca", "sao bernardo do campo", "sao paul…
$ customer_state           <chr> "SP", "SP", "SP", "SP", "SP", "SC", "SP", "M…


customer_id,n
<chr>,<int>


customer_unique_id,n
<chr>,<int>
8d50f5eadf50201ccdcedfb9e2ac8455,17
3e43e6105506432c953e165fb2acf44c,9
1b6c7548a2a1f9037c1fd3ddfed95f33,7
6469f99c1f9dfae7733b25662e7f1782,7
ca77025e7201e3b30c44b472ff346268,7
12f5d6e1cbf93dafd9dcc19095df0b3d,6
47c1a3033b8b77b3ab6e109eb4d5fdf3,6
63cfc61cee11cbe306bff5857d00bfe4,6
dc813062e0fc23409cd255f7f53c7074,6


customer_state,n
<chr>,<int>
SP,41746
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637
BA,3380
DF,2140
ES,2033


## Produtos

In [12]:
glimpse(products)

products %>% 
    count(product_category_name, sort = T)

Observations: 32,951
Variables: 9
$ product_id                 <chr> "1e9e8ef04dbcff4541ed26657ea517e5", "3aa07…
$ product_category_name      <chr> "perfumaria", "artes", "esporte_lazer", "b…
$ product_name_lenght        <int> 40, 44, 46, 27, 37, 60, 56, 56, 57, 36, 54…
$ product_description_lenght <int> 287, 276, 250, 261, 402, 745, 1272, 184, 1…
$ product_photos_qty         <int> 1, 1, 1, 1, 4, 1, 4, 2, 1, 1, 1, 4, 3, 2, …
$ product_weight_g           <int> 225, 1000, 154, 371, 625, 200, 18350, 900,…
$ product_length_cm          <int> 16, 30, 18, 26, 20, 38, 70, 40, 27, 17, 16…
$ product_height_cm          <int> 10, 18, 9, 4, 17, 5, 24, 8, 13, 10, 10, 19…
$ product_width_cm           <int> 14, 20, 15, 26, 13, 11, 44, 40, 17, 12, 16…


product_category_name,n
<chr>,<int>
cama_mesa_banho,3029
esporte_lazer,2867
moveis_decoracao,2657
beleza_saude,2444
utilidades_domesticas,2335
automotivo,1900
informatica_acessorios,1639
brinquedos,1411
relogios_presentes,1329


## Pedidos

In [13]:
glimpse(orders)

orders %>% 
  count(order_status, sort = T)

orders %>% 
  mutate_at(vars(4:8), lubridate::as_datetime) %>% 
  mutate(tempo_aprovacao_entrega = difftime(order_delivered_customer_date, order_approved_at, units = "days") %>% 
           as.numeric() %>% 
           floor(),
         tempo_estimado_realizado = difftime(order_estimated_delivery_date, order_delivered_customer_date, units = "days") %>% 
           as.numeric() %>% 
           floor()) %>% 
  as_tibble() %>% 
  summarise(Tempo_medio_aprovacao_entrega = mean(tempo_aprovacao_entrega, na.rm = T),
            Tempo_medio_estimado_realizado = mean(tempo_estimado_realizado, na.rm = T))

Observations: 99,441
Variables: 8
$ order_id                      <chr> "e481f51cbdc54678b7cc49136f2d6af7", "53…
$ customer_id                   <chr> "9ef432eb6251297304e76186b10a928d", "b0…
$ order_status                  <chr> "delivered", "delivered", "delivered", …
$ order_purchase_timestamp      <chr> "2017-10-02 10:56:33", "2018-07-24 20:4…
$ order_approved_at             <chr> "2017-10-02 11:07:15", "2018-07-26 03:2…
$ order_delivered_carrier_date  <chr> "2017-10-04 19:55:00", "2018-07-26 14:3…
$ order_delivered_customer_date <chr> "2017-10-10 21:25:13", "2018-08-07 15:2…
$ order_estimated_delivery_date <chr> "2017-10-18 00:00:00", "2018-08-13 00:0…


order_status,n
<chr>,<int>
delivered,96478
shipped,1107
canceled,625
unavailable,609
invoiced,314
processing,301
created,5
approved,2


Tempo_medio_aprovacao_entrega,Tempo_medio_estimado_realizado
<dbl>,<dbl>
11.64297,10.87688


## Pagamento

In [14]:
summary(payments$payment_value)

payments %>% 
  count(payment_type, sort = T)


    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    0.00    56.79   100.00   154.10   171.84 13664.08 

payment_type,n
<chr>,<int>
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


## Reviews

In [15]:
glimpse(reviews) 

summary(reviews$review_score)

reviews %>% 
  count(review_score, sort = T)

Observations: 100,000
Variables: 7
$ review_id               <chr> "7bc2406110b926393aa56f80a40eba40", "80e641a1…
$ order_id                <chr> "73fc7af87114b39712e6da79b0a377eb", "a548910a…
$ review_score            <int> 4, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 4, 4, 3, …
$ review_comment_title    <chr> "", "", "", "", "", "", "", "", "", "recomend…
$ review_comment_message  <chr> "", "", "", "Recebi bem antes do prazo estipu…
$ review_creation_date    <chr> "2018-01-18 00:00:00", "2018-03-10 00:00:00",…
$ review_answer_timestamp <chr> "2018-01-18 21:46:59", "2018-03-11 03:05:13",…


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   4.000   5.000   4.071   5.000   5.000 

review_score,n
<int>,<int>
5,57420
4,19200
1,11858
3,8287
2,3235


## Vendedores

In [16]:
glimpse(sellers)

sellers %>% 
  count(seller_state, sort = T)

Observations: 3,095
Variables: 4
$ seller_id              <chr> "3442f8959a84dea7ee197c632cb2df15", "d1b65fc7d…
$ seller_zip_code_prefix <int> 13023, 13844, 20031, 4195, 12914, 20920, 55325…
$ seller_city            <chr> "campinas", "mogi guacu", "rio de janeiro", "s…
$ seller_state           <chr> "SP", "SP", "RJ", "SP", "SP", "RJ", "PE", "SP"…


seller_state,n
<chr>,<int>
SP,1849
PR,349
MG,244
SC,190
RJ,171
RS,129
GO,40
DF,30
ES,23


# Análise de basket

In [81]:
dados_basket <- orders %>% 
  left_join(items, by = "order_id") %>% 
  left_join(products, by = "product_id") %>% 
  as_tibble() %>% 
  distinct(order_id, product_category_name) %>% 
  filter(product_category_name != "") %>% 
  group_by(order_id) %>% 
  filter(n() > 1) %>% 
  ungroup() %>% 
  data.table::as.data.table()

# Transformando em lista por transações, para realizar basket analysis

lista_transacoes <- split(dados_basket[["product_category_name"]], dados_basket[["order_id"]])

# require(arules)

lista_transacoes <- lista_transacoes %>% 
    as("transactions")

# Minerando regras de associação 

regras <- apriori(lista_transacoes, parameter = list(support = 0.0005, confidence = 0.025,
                                          minlen = 2, maxlen = 2))



regras

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
      0.025    0.1    1 none FALSE            TRUE       5   5e-04      2
 maxlen target   ext
      2  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[61 item(s), 727 transaction(s)] done [0.00s].
sorting and recoding items ... [61 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2

Warning message in apriori(lista_transacoes, parameter = list(support = 5e-04, confidence = 0.025, :
“Mining stopped (maxlen reached). Only patterns up to a length of 2 returned!”

 done [0.00s].
writing ... [355 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


set of 355 rules 

In [82]:
redundant <- which(colSums(is.subset(regras, regras)) > 1)
regras <- regras[-redundant]

In [83]:
regras

set of 109 rules 

In [85]:

# 
basket <- regras %>% 
  as("data.frame")

basket <- basket %>% 
  tidyr::separate(rules, into = c("Antecedente", "Consequente"), sep = "=>") %>% 
  mutate_at(vars(1:2), function(x){stringr::str_remove_all(x, "\\{") %>% 
      stringr::str_remove_all("\\}")}) %>% 
  arrange(-lift, -support, -confidence) %>%
  as_tibble()

basket


Antecedente,Consequente,support,confidence,lift,count
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
pc_gamer,cool_stuff,0.001375516,1.0000000,11.015152,1
fashion_calcados,bebes,0.002751032,1.0000000,7.817204,2
sinalizacao_e_seguranca,bebes,0.001375516,1.0000000,7.817204,1
artes_e_artesanato,brinquedos,0.001375516,0.5000000,7.270000,1
dvds_blu_ray,informatica_acessorios,0.001375516,0.5000000,7.127451,1
climatizacao,informatica_acessorios,0.001375516,0.5000000,7.127451,1
cine_foto,cool_stuff,0.001375516,0.5000000,5.507576,1
climatizacao,esporte_lazer,0.001375516,0.5000000,5.425373,1
cine_foto,esporte_lazer,0.001375516,0.5000000,5.425373,1
